In [1]:
!pip install pydub # requires ffmpeg

In [5]:
!pip install audio_extract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 71.1 MB/s eta 0:00:00a 0:00:01


In [6]:
# from pydub import AudioSegment
import os
import math
from audio_extract import extract_audio 

## Split the Audio in 10 Minute Segments

In [3]:
# Load the MP3 file
audio_file = AudioSegment.from_mp3("./data/sound/Building_Multimodal_Models.mp3")

# Calculate the number of 10-minute segments
num_segments = math.ceil(len(audio_file) / (60 * 1000 * 10))  # 60 seconds per minute, 1000 milliseconds per second * 10 for 10 mins

In [17]:
num_segments

11

In [18]:
# Create a new directory to store the output files
os.makedirs('./data/sound/output', exist_ok=True)

In [3]:
from pydub import AudioSegment
next_js_conf = AudioSegment.from_mp3("./data/sound/nextjs.conf/nextjs.conf.14.mp3")

/opt/conda/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
/opt/conda/lib/python3.11/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [Errno 2] No such file or directory: 'ffprobe'

In [20]:
for i in range(num_segments):
    start_time = i * 60000 * 10  # 60 seconds per minute, 1000 milliseconds per second
    end_time = (i + 1) * 60000 * 10
    segment = audio_file[start_time:min(end_time, len(audio_file))]
    segment.export(f"./data/sound/output/segment_{i+1}.mp3", format="mp3")

In [4]:
len(audio_file)

6470008

## Initialise Whisper

In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)




In [2]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=25,
    batch_size=16,
    torch_dtype=torch_dtype,
    device=device,
    generate_kwargs={"language": "en", "task": "transcribe", "max_new_tokens": 128},
)



## Process the Audio Segments and Store Documents for RAG

In [4]:
path = './data/sound/output'

In [11]:
for file_name in os.listdir(path):
    if file_name.endswith(".mp3"):
        
        audio_file_path = f"{path}/{file_name}"
        print(audio_file_path)
        result = pipe(audio_file_path)
        with open(f"./data/sound/output/{file_name[:-4]}.txt", "w") as f:
             f.write(result["text"])

./data/sound/output/segment_10.mp3
./data/sound/output/segment_5.mp3
./data/sound/output/segment_11.mp3
./data/sound/output/segment_1.mp3
./data/sound/output/segment_2.mp3
./data/sound/output/segment_4.mp3
./data/sound/output/segment_6.mp3
./data/sound/output/segment_3.mp3


/opt/conda/lib/python3.11/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


./data/sound/output/segment_7.mp3


/opt/conda/lib/python3.11/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


./data/sound/output/segment_8.mp3


/opt/conda/lib/python3.11/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


./data/sound/output/segment_9.mp3


/opt/conda/lib/python3.11/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


## Utilise RAG to Query the Docs

In [4]:
text = ""
path = './data/sound/output'
files = os.listdir(path)
files.sort(key=lambda x: int(x.split("_")[1][:-4])) # sort by segment number

for file_name in files:
    if file_name.endswith(".txt"):
        with open(f"{path}/{file_name}", "r") as f:
            text += f.read()


In [5]:
len(text.split())

17743

In [1]:
with open(f"{path}/text_full.txt", "w") as f:
    f.write(text)


NameError: name 'path' is not defined

## Process Without Splitting .mp3

Extract audio from the MOVs that we record

In [13]:
extract_audio(input_path="./data/transcribe_se6_vid/Screen Recording 2024-11-22 at 15.59.15.mov", output_path="./data/transcribe_se6_vid/audio3.mp3")

Success : audio file has been saved to "/home/jovyan/work/data/transcribe_se6_vid/audio3.mp3".


In [3]:
result = pipe('./data/transcribe_se6_vid/audio3.mp3')

/opt/conda/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [4]:
result

{'text': " Everything in the new Stream i6 is very similar to what you already know and have used in Stream i5.5. There are a few subtle differences that make the interface much more streamlined. First, on the right-hand side, you can see the feed tree. Instead of having buttons for context actions, we have placed them in a context menu that you can view by clicking on an item with the right click. Here you can edit, rename, archive, clone and other actions on the particular feed. Selecting an item from your dashboard you can go and quickly edit and create new feed from this item. Here is one major difference that now the completion of each task or the production of each feed in New Stream i has been broken down into steps and visualized by this wizard on top. You can select different options and then go next to select to proceed to a new step. Enter a feed name here to save your progress. Select size and click next. Here you can review preview what the banner would look like or you co

We will split this in 10 equal segments. Lets write it down first so we can load it later if needed from a text file

In [4]:
# write the text
with open("./data/sound/output/test-encode.txt", "w") as f:
    f.write(result['text'])

In [5]:
# load the text so we dont process every time
with open("./data/sound/nextjs.conf/text.txt", "r") as f:
    text = f.read()

In [1]:
with open("./data/sound/output/test-encode.txt", "r") as f:
    text = f.read()

In [2]:
arr = text.split()
segments = 2
text_segments = []
segment_length = len(arr) // segments
for i in range(segments):
    text_segments.append(" ".join(arr[i*segment_length:(i+1)*segment_length]))

len(text_segments[1].split()) * 1.5 # appx tokens / segment

3058.5

#### Can We Create a Function to Adjust Segments from Context Window

The function called gen_seg will take a context window, calculate with some margin of error number of words for the tokens in the context window and then split the text into segments of these many words, making sure not to finish a sentence abruptly. So, it will search inside the word allowance the last sentence and push it into the total segments.

lets get to it

In [2]:
def gen_seg(text, context_window, ratio=1.4):
    """
    Splits text into segments that should fit into a context window tokens
    Args:
        text: string, the text to split, 
        context_window: int, the number of tokens in the context window
        ratio: int, the ratio between tokens and words - normally each word converts into 1.4 tokens? adjustable.
    Returns:
        array of text_segments to use for generating summary of text
    """
    text_segments = []
    words_per_seg = int(context_window // ratio)
    remaining_words = text.split()
    # current_segment_num = 0

    while len(remaining_words) > words_per_seg:
        current_seg_arr = remaining_words[:words_per_seg]
        current_seg = " ".join(current_seg_arr)
        current_seg_sent = ".".join(current_seg.split('.')[:-1]) + "."
        remaining_words = text.split(current_seg_sent)[1].split()
        text_segments.append(current_seg_sent)
        
    text_segments.append(" ".join(remaining_words))        
    return text_segments

segs = gen_seg(text, context_window=3500)

# function to get all elements but the last in array
# def all_but_last(arr):
#     return arr[:-1]

In [3]:
len(segs)

2

## The Cumulative Summary Idea
Now that we have this in segments, lets design a prompt that would generate an accumulative summary tally for each segment.
Maybe the prompt can be something like this.

In [4]:
# lets see this in langsmith
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [5]:
prompt_instruction = """You are a professional writer. Your job here is to produce great, detailed and coherent summary of a document.
In order for the document to fit into your context window it is split into {number_of_parts} parts. Each part will be supplied to you as 
user prompt, together with the current key points up to this stage.

Do not repeat yourself in the summary, but rather add to it or rephrase it as you see fit, enhancing the key take away
points with new information. Only take out information that you do not think contributes to the main theme discussed in the document.

Supply updated key points list that you have so far, based on existing points and the current step copy.

When you receive copy for step - {number_of_parts} - please rework and rephrase what you have currently to produce a final list of up to 25 key points, 
followed by a short paragraph summarising the main theme of the entire document.
"""

# prompt_instruction_initial = """You are a professional writer. Your job here is to produce great, detailed and coherent summary of a document.
# In order for the document to fit into your context window it is split into {number_of_parts} parts. Each part will be supplied to you as 
# user prompt, together with the current key points up to this stage.

# Add a summary of the user prompt to the current summary and produce a new list.

# When you receive copy for step - {number_of_parts} - please rework and rephrase what you have currently to produce a final list of up to 25 key points.
# """

prompt_instruction_summary = """You are a professional writer. Your job here is to produce great, detailed and coherent summary of a document as a list
of key points. In order for the document to fit into your context window it is split into {number_of_parts} parts. Each part will be supplied to you as 
user prompt, together with the current key points up to this stage.

You are to summarise the following user prompt and add it in form of key points to the current list provided under the tag <key_points>. The new list
should look like this:
<key_points>
1. Some example key point
2. Another key point
3. Key point from the latest user prompt
4. More points from the latest prompt
5. ...
6. and so on...

"""

prompt_user = """The key points thus far are:
{{summary}}
The copy of step number {current} follows below:
{input}
"""

prompt_user_key_points = """The key points thus far are:
<key_points>
{{summary}}
The copy to summarise into key points and add to the above list is:
{input}
Please when supplying the new list of total key points, remove any repeating ideas or themes and rephrase them for clarity.
<key_points>
"""

initial_user = """The key points thus far are:
None. This is the initial input.
The copy of step number 1 follows below:
{input}
"""
prompt_initial_user_new = """The key points thus far are:
None. This is the initial input.
The copy to summarise as 10 key points follows below:
{input}
<key_points>
"""

template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>{instruction}
<|eot_id|><|start_header_id|>user <|end_header_id|>{user}
<|eot_id|><|start_header_id|>assistant <|end_header_id|>
"""

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
import torch
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    GenerationConfig,
    pipeline
)

### Initial Step

We need this in order to supply the first input to the chain of chains. For each step we will generate the relative instruction.

In [7]:
initial_instruction = prompt_instruction_summary.format(number_of_parts=len(segs))
template_with_instr = template.format(instruction=initial_instruction, user=prompt_initial_user_new)

In [8]:
# template_initial = template_with_instr.format(current=1)
prompt = PromptTemplate.from_template(template_with_instr)
prompt

PromptTemplate(input_variables=['input'], template='\n<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a professional writer. Your job here is to produce great, detailed and coherent summary of a document as a list\nof key points. In order for the document to fit into your context window it is split into 2 parts. Each part will be supplied to you as \nuser prompt, together with the current key points up to this stage.\n\nYou are to summarise the following user prompt and add it in form of key points to the current list provided under the tag <key_points>. The new list\nshould look like this:\n<key_points>\n1. Some example key point\n2. Another key point\n3. Key point from the latest user prompt\n4. More points from the latest prompt\n5. ...\n6. and so on...\n\n\n<|eot_id|><|start_header_id|>user <|end_header_id|>The key points thus far are:\nNone. This is the initial input.\nThe copy to summarise as 10 key points follows below:\n{input}\n<key_points>\n\n<|eot_id|><|st

In [8]:
# partial = prompt.partial(input=text_segments[0])

In [9]:
model_name = "../ext_models/Meta-Llama-3-8B-Instruct"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [10]:
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, 
                                             device_map=DEVICE, 
                                             torch_dtype="auto")
generation_config = GenerationConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
stop_token = "<|eot_id|>"  
stop_token_id = tokenizer.encode(stop_token)[0]
begin_token = "<|begin_of_text|>"
begin_token_id = tokenizer.encode(begin_token)[0]
generation_config.eos_token_id = stop_token_id
generation_config.begin_token_id = begin_token_id
generation_config.max_new_tokens = 1200
generation_config.temperature = 0.1
generation_config.top_p = 0.90
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15
generation_config.pad_token_id = tokenizer.eos_token_id

In [12]:
!pip install langchain_huggingface

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain.chains import SimpleSequentialChain, SequentialChain, LLMChain

In [14]:
llm_pipeline = pipeline("text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        generation_config=generation_config, 
                        return_full_text=False) 
llm = HuggingFacePipeline(pipeline=llm_pipeline)

In [15]:
initial_chain = LLMChain(llm=llm, prompt=prompt, output_parser=StrOutputParser())
# initial_chain = prompt | llm | StrOutputParser()

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [16]:
chains = [initial_chain]

In [17]:
for i in range(len(segs)-1):
    template_with_instr = template.format(instruction=prompt_instruction_summary.format(number_of_parts=len(segs)), user=prompt_user_key_points)
    template_internal = template_with_instr.format(input=segs[i+1], current=i+2)
    prompt = PromptTemplate.from_template(template_internal)
    internal_chain = LLMChain(llm=llm, prompt=prompt, output_parser=StrOutputParser())
    chains.append(internal_chain)
  

In [18]:
chains[1], len(chains)

(LLMChain(prompt=PromptTemplate(input_variables=['summary'], template="\n<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a professional writer. Your job here is to produce great, detailed and coherent summary of a document as a list\nof key points. In order for the document to fit into your context window it is split into 2 parts. Each part will be supplied to you as \nuser prompt, together with the current key points up to this stage.\n\nYou are to summarise the following user prompt and add it in form of key points to the current list provided under the tag <key_points>. The new list\nshould look like this:\n<key_points>\n1. Some example key point\n2. Another key point\n3. Key point from the latest user prompt\n4. More points from the latest prompt\n5. ...\n6. and so on...\n\n\n<|eot_id|><|start_header_id|>user <|end_header_id|>The key points thus far are:\n<key_points>\n{summary}\nThe copy to summarise into key points and add to the above list is:\nIt's not a deal

In [19]:
chain_of_chains = SimpleSequentialChain(chains=chains, verbose=True)

In [20]:
result = chain_of_chains.run(input=segs[0])

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
Here are the key points summarized:

1. The author purchased the M3 Pro MacBook Pro and has been using it daily for three months, sharing his experiences and observations.
2. The M3 Pro has a unique set of upgrades and downgrades compared to the previous generation, making it difficult to understand its capabilities.
3. The author emphasizes the importance of considering real-world usage rather than focusing solely on specifications and features.
4. He notes that the M3 Pro's 11-core CPU, 14-core GPU, 512GB storage, and 18GB RAM provide excellent performance for everyday tasks.
5. Battery life has improved significantly, with the author achieving up to 16 hours of use on a single charge, a 20-30% increase over the M2 Pro.
6. The author addresses common concerns about leaving the MacBook plugged in continuously, assuring that it won't degrade the battery.
7. External storage issues, such as disconnections when the Mac goes to sleep, can be

In [21]:
print(result)

Here is the updated list of key points:

<key_points>
1. The author purchased the M3 Pro MacBook Pro and has been using it daily for three months, sharing his experiences and observations.
2. The M3 Pro has a unique set of upgrades and downgrades compared to the previous generation, making it difficult to understand its capabilities.
3. The author emphasizes the importance of considering real-world usage rather than focusing solely on specifications and features.
4. He notes that the M3 Pro's 11-core CPU, 14-core GPU, 512GB storage, and 18GB RAM provide excellent performance for everyday tasks.
5. Battery life has improved significantly, with the author achieving up to 16 hours of use on a single charge, a 20-30% increase over the M2 Pro.
6. The author addresses common concerns about leaving the MacBook plugged in continuously, assuring that it won't degrade the battery.
7. External storage issues, such as disconnections when the Mac goes to sleep, can be resolved by using a powered hu